<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2022</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Типы - Часть 2</b><br/>
    <br/>
    <font>Никита Бондарцев</font><br/>
</center>

### Что мы сегодня разберем?

### Специальный тип Any

In [1]:
import typing as tp
print(tp.Any.__doc__.split("\n")[0])

Special type indicating an unconstrained type.


### Почему тип Any специальный?

In [2]:
import typing as tp

# Тип-класс
class A:
    pass

# Тип-класс
B = int

# Тип-объект
C = tp.Any

type(A), type(B), type(C)

(type, type, typing._SpecialForm)

In [3]:
isinstance(1, tp.Any)

TypeError: typing.Any cannot be used with isinstance()

### Специальный тип Union

In [4]:
import typing as tp
print(tp.Union.__doc__.split("\n")[0])

Union type; Union[X, Y] means either X or Y.


Определение подтипа:
* ∀ A: A -> A
* A -> B  =>  A.values ⊇ B.values
* A -> B  =>  A.functions ⊆ B.functions

tp.Union[float, str] => объединение всех значений, пересечение всех методов, поэтому

#### Выполняются ли такие свойства?

tp.Union[float, str] -> float \
tp.Union[float, str] -> str

In [6]:
%%typecheck

import typing as tp

class A:
    def am(self) -> None:
        pass
    
    def run(self) -> None:
        pass

class B:
    def run(self) -> None:
        pass


a: tp.Union[A, B] = A()
reveal_type(a)
a.am()
a.run()

<string>:18: note: Revealed type is "Union[__main__.A, __main__.B]"
<string>:19: error: Item "B" of "Union[A, B]" has no attribute "am"
Found 1 error in 1 file (checked 1 source file)



In [16]:
tp.Union[float, str] == tp.Union[str, float], tp.Union[str, float] == str | float

(True, True)

In [8]:
float == tp.Union[float]

True

### Union → Примеры 1/2

In [11]:
%%typecheck
# Использование надтипов в Union

import typing as tp

def f(a: tp.Union[float, str]):
    pass

f(2)
f(2.0)
f("hello")
f({})

<string>:12: error: Argument 1 to "f" has incompatible type "Dict[<nothing>, <nothing>]"; expected "Union[float, str]"
Found 1 error in 1 file (checked 1 source file)



In [10]:
%%typecheck
# Использование подтипов в Union

import typing as tp

def f(a: tp.Union[int, str]):
    pass

f(2)
f(2.0)
f("hello")
f({})

<string>:10: error: Argument 1 to "f" has incompatible type "float"; expected "Union[int, str]"
<string>:12: error: Argument 1 to "f" has incompatible type "Dict[<nothing>, <nothing>]"; expected "Union[int, str]"
Found 2 errors in 1 file (checked 1 source file)



### Union → Примеры 2/2

In [12]:
%%typecheck
# Вывод типов с учетом isinstance


import typing as tp

def f(a: int | float | str):
    reveal_type(a)
    if isinstance(a, int):
        reveal_type(a)
    else:
        reveal_type(a)
        if isinstance(a, str):
            reveal_type(a)
        else:
            reveal_type(a)


<string>:8: note: Revealed type is "Union[builtins.int, builtins.float, builtins.str]"
<string>:10: note: Revealed type is "builtins.int"
<string>:12: note: Revealed type is "Union[builtins.float, builtins.str]"
<string>:14: note: Revealed type is "builtins.str"
<string>:16: note: Revealed type is "builtins.float"
Success: no issues found in 1 source file



In [13]:
%%typecheck
# Вывод типов с учетом isinstance
# --warn-unreachable

import typing as tp

def f(a: int | float | str):
    reveal_type(a)
    if isinstance(a, float):
        reveal_type(a)
    else:
        reveal_type(a)
        if isinstance(a, str):
            reveal_type(a)
        else:
            reveal_type(a)

<string>:8: note: Revealed type is "Union[builtins.int, builtins.float, builtins.str]"
<string>:10: note: Revealed type is "builtins.float"
<string>:12: note: Revealed type is "builtins.str"
<string>:14: note: Revealed type is "builtins.str"
Success: no issues found in 1 source file



### Специальный тип Optional

In [17]:
import typing as tp
print(tp.Optional.__doc__.split("\n")[0])

Optional type.


In [18]:
import typing as tp

tp.Optional[float] == tp.Union[float, None], tp.Optional[float] == float | None

(True, True)

### Специальный тип Optional, примеры 1/1

In [19]:
%%typecheck

import typing as tp

def f(a: float | None):
    pass

f(1)
f(1.0)
f("1")
f(None)

<string>:10: error: Argument 1 to "f" has incompatible type "str"; expected "Optional[float]"
Found 1 error in 1 file (checked 1 source file)



In [20]:
%%typecheck
# reveal в Optional делается аналогично Union

import typing as tp

def f(a: float | None):
    reveal_type(a)
    if a is None:
        reveal_type(a)
    else:
        reveal_type(a)

<string>:7: note: Revealed type is "Union[builtins.float, None]"
<string>:9: note: Revealed type is "None"
<string>:11: note: Revealed type is "builtins.float"
Success: no issues found in 1 source file



### Что такое генерики?

### Ковариантность/контрвариантность

### Примеры вариантности

### typing.Tuple (tuple)

In [21]:
import typing as tp
print(tp.Tuple.__doc__.split("\n")[0])

Tuple type; Tuple[X, Y] is the cross-product type of X and Y.


In [22]:
%%typecheck
import typing as tp

a: tp.Tuple[int, str] = (1, "hello")
b: tuple[int, int, int] = (1, 2, 3)
c: tuple[int, ...] = (1, 2, 3, 4, 5)

Success: no issues found in 1 source file



### Tuple, пример

In [23]:
%%typecheck
# Heterogeneus tuple 



def f(a: tuple[int, float]):
    a[0] << 10
    a[1] << 10
    a[2] + 1

f((1, 1.4))
f((1, 1.4, 1))
f((1, 1))
f((1.4, 1))
f((1, "1"))

<string>:8: error: Unsupported operand types for << ("float" and "int")
<string>:9: error: Tuple index out of range
<string>:12: error: Argument 1 to "f" has incompatible type "Tuple[int, float, int]"; expected "Tuple[int, float]"
<string>:14: error: Argument 1 to "f" has incompatible type "Tuple[float, int]"; expected "Tuple[int, float]"
<string>:15: error: Argument 1 to "f" has incompatible type "Tuple[int, str]"; expected "Tuple[int, float]"
Found 5 errors in 1 file (checked 1 source file)



In [24]:
%%typecheck
# Homogeneus tuple 

import typing as tp

def f(a: tp.Tuple[float, ...]):
    a[0] + 1
    a[1] << 10
    a[100500] = 1.54

f((1, 2, 3, 4, 5))
f((1.1, 2.1, 3.1, 4.1, 5.1))
f(("3", "2", "1"))
f(tuple())

<string>:8: error: Unsupported operand types for << ("float" and "int")
<string>:9: error: Unsupported target for indexed assignment ("Tuple[float, ...]")
<string>:13: error: Argument 1 to "f" has incompatible type "Tuple[str, str, str]"; expected "Tuple[float, ...]"
Found 3 errors in 1 file (checked 1 source file)



### Какая вариантность у Tuple?

### typing.List (list)

In [25]:
import typing as tp
print("\n".join(tp.List.__doc__.split("\n")[0:3]))

A generic version of list.


### List, пример 1/3

In [26]:
%%typecheck

a = [1, "hello", 5.1]
reveal_type(a)

a.append(1)
a.append(1.0)
a.append("hi")
a.append([])

<string>:5: note: Revealed type is "builtins.list[builtins.object]"
Success: no issues found in 1 source file



In [27]:
%%typecheck

a = []
reveal_type(a)

<string>:5: error: Need type annotation for "a" (hint: "a: List[<type>] = ...")
<string>:6: note: Revealed type is "builtins.list[Any]"
Found 1 error in 1 file (checked 1 source file)



### List, пример 2/3

In [28]:
%%typecheck

a: list[float] = []

a.append(1)
reveal_type(a)

<string>:8: note: Revealed type is "builtins.list[builtins.float]"
Success: no issues found in 1 source file



In [29]:
%%typecheck

a: list[int] = []

a.append(1.1)
reveal_type(a)

<string>:7: error: Argument 1 to "append" of "list" has incompatible type "float"; expected "int"
<string>:8: note: Revealed type is "builtins.list[builtins.int]"
Found 1 error in 1 file (checked 1 source file)



### List, пример 3/3

In [30]:
%%typecheck
# Повышение типа параметра

def foo(a: list[int]) -> None:
    pass

my_list = [1.1, 3.1, 5.1]
reveal_type(my_list)

foo(my_list)

<string>:10: note: Revealed type is "builtins.list[builtins.float]"
<string>:12: error: Argument 1 to "foo" has incompatible type "List[float]"; expected "List[int]"
Found 1 error in 1 file (checked 1 source file)



In [31]:
%%typecheck
# Понижение типа параметра

def foo(a: list[float]) -> None:
    pass

my_list = [1, 2, 3]
reveal_type(my_list)

foo(my_list)

<string>:10: note: Revealed type is "builtins.list[builtins.int]"
<string>:12: error: Argument 1 to "foo" has incompatible type "List[int]"; expected "List[float]"
<string>:12: note: "List" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:12: note: Consider using "Sequence" instead, which is covariant
Found 1 error in 1 file (checked 1 source file)



### Какая вариантность у List?

### typing.Sequence/typing.Mapping

### Sequence, пример

In [32]:
%%typecheck



def foo(a: list[float]) -> float:
    return a[0]

my_list = [1, 3, 5]

foo(my_list)

<string>:10: error: Argument 1 to "foo" has incompatible type "List[int]"; expected "List[float]"
<string>:10: note: "List" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:10: note: Consider using "Sequence" instead, which is covariant
Found 1 error in 1 file (checked 1 source file)



In [33]:
%%typecheck

import typing as tp

def foo(a: tp.Sequence[float]) -> float:
    return a[0]

my_list = [1, 3, 5]

foo(my_list)

Success: no issues found in 1 source file



### Mapping, пример

In [34]:
%%typecheck



def foo(a: dict[str, float]) -> float:
    return a["key"]

my_dict = {"hey": 1}

foo(my_dict)

<string>:10: error: Argument 1 to "foo" has incompatible type "Dict[str, int]"; expected "Dict[str, float]"
<string>:10: note: "Dict" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:10: note: Consider using "Mapping" instead, which is covariant in the value type
Found 1 error in 1 file (checked 1 source file)



In [35]:
%%typecheck

import typing as tp

def foo(a: tp.Mapping[str, float]) -> float:
    return a["key"]

my_dict = {"hey": 1}

foo(my_dict)

Success: no issues found in 1 source file



### Иерархия типов генериков
<img src="images/hierarchy2.jpg">

### Примеры на иерархию Sequence/Mapping 

In [38]:
%%typecheck

import typing as tp


def foo1(a: tp.Sequence[float]) -> None:
    pass

def foo2(a: tp.MutableSequence[float]) -> None:
    pass

def foo3(a: list[float]) -> None:
    pass

def foo4(a: list) -> None:
    pass


a = [1]
foo1(a)
foo2(a)
foo3(a)
foo4(a)


<string>:21: error: Argument 1 to "foo2" has incompatible type "List[int]"; expected "MutableSequence[float]"
<string>:22: error: Argument 1 to "foo3" has incompatible type "List[int]"; expected "List[float]"
<string>:22: note: "List" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:22: note: Consider using "Sequence" instead, which is covariant
Found 2 errors in 1 file (checked 1 source file)



In [39]:
%%typecheck

import typing as tp


def foo1(a: tp.Mapping[str, float]) -> None:
    pass

def foo2(a: tp.MutableMapping[str, float]) -> None:
    pass

def foo3(a: dict[str, float]) -> None:
    pass

def foo4(a: dict) -> None:
    pass


a = {'a': 1}
foo1(a)
foo2(a)
foo3(a)
foo4(a)

<string>:21: error: Argument 1 to "foo2" has incompatible type "Dict[str, int]"; expected "MutableMapping[str, float]"
<string>:22: error: Argument 1 to "foo3" has incompatible type "Dict[str, int]"; expected "Dict[str, float]"
<string>:22: note: "Dict" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:22: note: Consider using "Mapping" instead, which is covariant in the value type
Found 2 errors in 1 file (checked 1 source file)



### Generic functions, TypeVar

In [40]:
import typing
tp.TypeVar.__doc__.split("\n")[0]

'Type variable.'

### TypeVar, пример 1/3 (правда пока что без TypeVar'а)

In [41]:
%%typecheck

import typing as tp


def foo(a: int, b: int) -> int:
    reveal_type(a)
    return a + b

a = foo(1, 2)
reveal_type(a)

<string>:7: note: Revealed type is "builtins.int"
<string>:11: note: Revealed type is "builtins.int"
Success: no issues found in 1 source file



In [42]:
%%typecheck

import typing as tp


def foo(a: int, b: int) -> int:
    return a + b

def foo2(a: str, b: str) -> str:
    return a + b

a = foo(1, 2)
reveal_type(a)

b = foo2("1", "2")
reveal_type(b)

c = foo("1", 2)
reveal_type(c)

<string>:13: note: Revealed type is "builtins.int"
<string>:16: note: Revealed type is "builtins.str"
<string>:18: error: Argument 1 to "foo" has incompatible type "str"; expected "int"
<string>:19: note: Revealed type is "builtins.int"
Found 1 error in 1 file (checked 1 source file)



### TypeVar, пример 2/3

In [43]:
%%typecheck

import typing as tp


def foo(a: tp.Union[int, str], b: tp.Union[int, str]) -> tp.Union[int, str]:
    reveal_type(a)
    return a + b

a = foo(1, 2)
reveal_type(a)

b = foo("1", "2")
reveal_type(b)

c = foo("1", 2)
reveal_type(c)


<string>:7: note: Revealed type is "Union[builtins.int, builtins.str]"
<string>:8: error: Unsupported operand types for + ("int" and "str")
<string>:8: error: Unsupported operand types for + ("str" and "int")
<string>:8: note: Both left and right operands are unions
<string>:11: note: Revealed type is "Union[builtins.int, builtins.str]"
<string>:14: note: Revealed type is "Union[builtins.int, builtins.str]"
<string>:17: note: Revealed type is "Union[builtins.int, builtins.str]"
Found 2 errors in 1 file (checked 1 source file)



In [44]:
%%typecheck

import typing as tp

T = tp.TypeVar('T', int, str)

def foo(a: T, b: T) -> T:
    reveal_type(a)
    return a + b

a = foo(1, 2)
reveal_type(a)

b = foo("1", "2")
reveal_type(b)

c = foo("1", 2)
reveal_type(c)

<string>:8: note: Revealed type is "builtins.int"
<string>:8: note: Revealed type is "builtins.str"
<string>:12: note: Revealed type is "builtins.int"
<string>:15: note: Revealed type is "builtins.str"
<string>:17: error: Value of type variable "T" of "foo" cannot be "object"
<string>:18: note: Revealed type is "builtins.object"
Found 1 error in 1 file (checked 1 source file)



### TypeVar, пример 3/3

In [45]:
%%typecheck

import typing as tp

T = tp.TypeVar('T')

def foo(a: list[T], n: int) -> T:
    return a[n]

a = foo([1, 2], 0)
reveal_type(a)

<string>:11: note: Revealed type is "builtins.int"
Success: no issues found in 1 source file



###  Переменные классов, Type

In [46]:
# https://www.python.org/dev/peps/pep-0484/#the-type-of-class-objects

import typing as tp
print(tp.Type.__doc__.split("\n")[0])

A special construct usable to annotate class objects.


In [47]:
%%typecheck

import typing as tp

a: tp.Type[int] = int

class B:
    pass

b: tp.Type[B] = B

Success: no issues found in 1 source file



###  Type, пример

In [48]:
%%typecheck
import typing as tp

class A:
    pass

class B(A):
    pass

def foo(a: tp.Type[A]) -> None:
    pass


foo(A())
foo(A)
foo(B())
foo(B)

<string>:14: error: Argument 1 to "foo" has incompatible type "A"; expected "Type[A]"
<string>:16: error: Argument 1 to "foo" has incompatible type "B"; expected "Type[A]"
Found 2 errors in 1 file (checked 1 source file)



In [49]:
%%typecheck
import typing as tp


class A:
    def __init__(self, a: int) -> None:
        self.a = a

    @classmethod
    def build(cls: tp.Type[A]) -> A:
        return cls(1)


reveal_type(A.build())

<string>:14: note: Revealed type is "__main__.A"
Success: no issues found in 1 source file



###  Какая вариантность у Type?

### Функциональный тип Callable

In [51]:
import typing as tp
print(tp.Callable.__doc__.split("\n")[0])

Callable type; Callable[[int], str] is a function of (int) -> str.


In [52]:
%%typecheck

import typing as tp

def g(a: int, b: float) -> float:
    return 1.1

a: tp.Callable[[int, float], float] = g

Success: no issues found in 1 source file



### Callable, пример 1/4

In [53]:
%%typecheck
# Привычная функция

import typing as tp

def f(a: tp.Callable[[int, float], float]) -> None:
    pass

def g(a: int, b: float) -> float:
    return 1.1

f(g)

Success: no issues found in 1 source file



In [54]:
%%typecheck
# Функция без аргументов

import typing as tp

def f(a: tp.Callable[[], float]):
    pass

def g() -> float:
    return 1.1

f(g)

Success: no issues found in 1 source file



### Callable, пример 2/4

In [55]:
%%typecheck
# Любые аргументы

import typing as tp

def f(a: tp.Callable[..., float]):
    pass

def g(a: int, b: str, c: int) -> float:
    return 1.1

f(g)

Success: no issues found in 1 source file



In [56]:
%%typecheck
# И лямбда тоже

import typing as tp

def f(a: tp.Callable[..., float]):
    pass

f(lambda a, b, c: 1.1)

Success: no issues found in 1 source file



### Callable, пример 3/4

In [57]:
%%typecheck
# Повышаем тип аргумента

import typing as tp

def f(a: tp.Callable[[int], float]):
    pass

def g(a: float) -> float:
    return a

f(g)

Success: no issues found in 1 source file



In [58]:
%%typecheck
# Понижаем тип аргумента

import typing as tp

def f(a: tp.Callable[[float], float]):
    pass

def g(a: int) -> float:
    return a

f(g)

<string>:12: error: Argument 1 to "f" has incompatible type "Callable[[int], float]"; expected "Callable[[float], float]"
Found 1 error in 1 file (checked 1 source file)



### Callable, пример 4/4

In [59]:
%%typecheck
# Повышаем тип возвращаемого значения

import typing as tp

def f(a: tp.Callable[[int], int]):
    pass

def g(a: int) -> float:
    return a

f(g)

<string>:12: error: Argument 1 to "f" has incompatible type "Callable[[int], float]"; expected "Callable[[int], int]"
Found 1 error in 1 file (checked 1 source file)



In [60]:
%%typecheck
# Понижаем тип возвращаемого значения

import typing as tp

def f(a: tp.Callable[[int], float]):
    pass

def g(a: int) -> int:
    return a

f(g)

Success: no issues found in 1 source file



### Какая вариантность Callable по аргументам? По возвращаемому значению?

### Создание собственных Generic классов

In [63]:
import typing as tp
print(tp.Generic.__doc__.split('\n')[0])

Abstract base class for generic types.


### Generic, пример 1/2

In [70]:
%%typecheck
import typing as tp
    
T = tp.TypeVar("T", str, int)

class A:
    def __init__(self, a: T) -> None:
        self._a = a
        reveal_type(self._a)
        
    def am(self) -> T:
        reveal_type(self._a)
        return self._a + self._a

    
a = A(1)
reveal_type(a)
b = a.am()
reveal_type(b)

<string>:8: error: Need type annotation for "_a"
<string>:9: note: Revealed type is "builtins.str"
<string>:9: note: Revealed type is "Any"
<string>:12: note: Revealed type is "Any"
<string>:17: note: Revealed type is "__main__.A"
<string>:19: note: Revealed type is "builtins.str"
Found 1 error in 1 file (checked 1 source file)



In [71]:
%%typecheck
import typing as tp

T = tp.TypeVar("T", int, str)

class A(tp.Generic[T]):    
    def __init__(self, a: T) -> None:
        self._a: T = a
        reveal_type(self._a)
        
    def am(self) -> T:
        reveal_type(self._a)
        return self._a


a = A(1)
reveal_type(a)
b = a.am()
reveal_type(b)

c = A("hello")
reveal_type(c)
d = c.am()
reveal_type(d)

<string>:9: note: Revealed type is "builtins.int"
<string>:9: note: Revealed type is "builtins.str"
<string>:12: note: Revealed type is "builtins.int"
<string>:12: note: Revealed type is "builtins.str"
<string>:17: note: Revealed type is "__main__.A[builtins.int]"
<string>:19: note: Revealed type is "builtins.int"
<string>:22: note: Revealed type is "__main__.A[builtins.str]"
<string>:24: note: Revealed type is "builtins.str"
Success: no issues found in 1 source file



### Generic, пример 2/2

In [6]:
%%typecheck

import typing as tp

T = tp.TypeVar("T")

class A(tp.Generic[T]):
    def __init__(self) -> None:
        self._a: tp.List[T] = []
        
    def add(self, a: T) -> None:
        self._a.append(a)


a: A[int] = A()
a.add(1)
reveal_type(a)

b: A[float] = A()
b.add("sss")

<string>:17: note: Revealed type is "__main__.A[builtins.int]"
<string>:20: error: Argument 1 to "add" of "A" has incompatible type "str"; expected "float"
Found 1 error in 1 file (checked 1 source file)



In [7]:
%%typecheck
# Что поправить, чтобы был success?

import typing as tp

T = tp.TypeVar("T", bound=int)

class A(tp.Generic[T]):
    def __init__(self, a: list[T]) -> None:
        self._a = a
        
    def get(self) -> list[T]:
        return self._a

    
def f(a: A[T]):
    pass

a: A[bool] = A([True, False])
f(a)

Success: no issues found in 1 source file



### Типы → Nominal subtyping

### Типы → Structural subtyping

### Structural subtyping, пример 1/2

In [10]:
%%typecheck
import typing as tp
# проставьте тайпинг, чтобы не упало

def validate_size(a, n) -> None:
    if (a_len := len(a)) > n:
        raise ValueError(
            f"Structure length {a_len} is greater then expected {n}"
        )

        
validate_size([10, 11], 1)

Success: no issues found in 1 source file



In [11]:
%%typecheck
import typing as tp
# проставьте тайпинг, чтобы не упало

def validate_size(a: tp.Sequence[int], n) -> None:
    if (a_len := len(a)) > n:
        raise ValueError(
            f"Structure length {a_len} is greater then expected {n}"
        )

        
validate_size([10, 11], 1)
validate_size((1, 3, 10), 1)


Success: no issues found in 1 source file



### Structural subtyping, пример 2/2

In [15]:
%%typecheck
import typing as tp
# проставьте тайпинг, чтобы не упало

def validate_size(a, n) -> None:
    if (a_len := len(a)) > n:
        raise ValueError(
            f"Structure length {a_len} is greater then expected {n}"
        )
        
validate_size([10, 11], 1)
validate_size((1, 3, 10), 1)
validate_size({1, 3, 10}, 1)

Success: no issues found in 1 source file



In [16]:
%%typecheck
import typing as tp
# проставьте тайпинг, чтобы не упало

def validate_size(a, n) -> None:
    if (a_len := len(a)) > n:
        raise ValueError(
            f"Structure length {a_len} is greater then expected {n}"
        )
        
validate_size([10, 11], 1)
validate_size((1, 3, 10), 1)
validate_size({1, 3, 10}, 1)
validate_size(A(), 1)

class A:
    def __len__(self):
        return 1

Success: no issues found in 1 source file



### Собственный Structural Subtping - Protocol

In [17]:
%%typecheck

import typing as tp


class A:
    def close(self):
        print("Close enough")

class B:
    pass


c = open("my_file")


def foo(a) -> None:
    reveal_type(a)
    a.close()


foo(A())
foo(B())
foo(c)

<string>:18: note: Revealed type is "Any"
Success: no issues found in 1 source file



In [18]:
%%typecheck

import typing as tp


class Closeable(tp.Protocol):
    def close(self) -> None:
        pass
    

class A:
    def close(self):
        print("Close enough")

class B:
    pass


c = open("my_file")


def foo(a: Closeable) -> None:
    reveal_type(a)
    a.close()


foo(A())
foo(B())
foo(c)

<string>:23: note: Revealed type is "__main__.Closeable"
<string>:28: error: Argument 1 to "foo" has incompatible type "B"; expected "Closeable"
Found 1 error in 1 file (checked 1 source file)



### Protocol and isinstance, runtime_checkable

In [21]:
import typing as tp

# @tp.runtime_checkable
class Closeable(tp.Protocol):
    def close(self) -> None:
        pass


class A:
    def close(self):
        print("Close enough")

isinstance(A(), Closeable)

TypeError: Instance and class checks can only be used with @runtime_checkable protocols

### Сложна: Generic + Protocol

In [23]:
%%typecheck

import typing as tp

def foo(s: tp.Iterable[int]) -> None:
    pass

class A:
    pass

T = tp.TypeVar("T")

class B(tp.Generic[T]):
    def __iter__(self) -> tp.Iterator[T]:
        return iter([])
    


foo(A())

b: B[int] = B()
foo(b)

c: B[str] = B()
foo(c)

foo([])

<string>:19: error: Argument 1 to "foo" has incompatible type "A"; expected "Iterable[int]"
<string>:25: error: Argument 1 to "foo" has incompatible type "B[str]"; expected "Iterable[int]"
Found 2 errors in 1 file (checked 1 source file)

